In [1]:
import pandas as pd
import requests
from datetime import date, timedelta
import json
from pymongo import MongoClient 


In [2]:
client = MongoClient() 
client = MongoClient("mongodb://localhost:27017")

In [3]:
mydatabase = client['Zones'] 
mycollection = mydatabase['NiftyNxt50W'] 

In [4]:
def fiblvlprice(lvl,ind):
    return (explosive['Low'][ind]-explosive['High'][ind])*lvl+explosive['High'][ind]


In [5]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [6]:
def isCandleExplosive(high,low,openn,close,atravg):
    mp = 1
    ratio = 20
    cand = 10
    rang = atravg

    candr = abs(high-low)
    bodyr = abs(openn-close)
    if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):
        return True
    else:
        return False

def get_mondays_fridays(a, b):

    start_date = pd.to_datetime(a)
    end_date = pd.to_datetime(b)

  # Ensure start_date is earlier or equal to end_date
    if start_date > end_date:
        start_date, end_date = end_date, start_date

  # Create a date range (optimized for efficiency)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

  # Filter for Mondays and Fridays using boolean indexing
    mondays = date_range[date_range.weekday == 0]
    fridays = date_range[date_range.weekday == 4]

    return mondays, fridays

In [7]:
def checkdata(df):
    ind = []
    date = []
    o = []
    h = []
    l = []
    c = []
    explosive = pd.DataFrame()

    for i in range(0,len(df['date'])-1):
        if isCandleExplosive(df['high'][i],df['low'][i],df['open'][i],df['close'][i],df['atravg'][i]):
            #print(df['date'][i],df['atr'][i])
            ind.append(i)
            date.append(df['date'][i])
            o.append(df['open'][i])
            h.append(df['high'][i])
            l.append(df['low'][i])
            c.append(df['close'][i])
    explosive['index'] = ind
    explosive['Date'] = date
    explosive['Open'] = o
    explosive['High'] = h
    explosive['Low'] = l
    explosive['Close'] = c
    return explosive

In [9]:

def fetchData(id,frm,to):
    url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/day?user_id=EM9701&oi=1&from="+frm+"&to="+to+""
    #url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/60minute?user_id=EM9701&oi=1&from=2023-11-23&to=2024-02-21"
    headers = {
        "accept": "*/*",
        "accept-language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": "enctoken i5j6ph2AVEEW6N6jN9akvtLjC4H69iH3xFZ4hhheRul25IoNvXdTef5MmvZKnNclzHtIJqzU0+c7DQ9uduQGlIbPzn9MkTzMxxs41YLQdhC57NJiYZPiag==",  # Replace with your actual authorization token
        "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin"
    }

    response = requests.get(url, headers=headers)

    # Check for successful response
    if response.status_code == 200:
        #data = response.json()  # Assuming response is JSON
        # Process the data here
        #print(data)
        return response.json()
    else:
        print(f"Error: {response.status_code}")

    

In [10]:
def createDffromData(data):
    column_names = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    df = pd.DataFrame(data['data']['candles'], columns=column_names)
    for i in range(0,len(df['open'])):
        df['date'][i]=df['date'][i].split('T')[0]
    return df

In [11]:
def putAtrValues(df):
    atri = atr(df,1)
    atr_avg = atr(df,20)
    df = df.merge(atr_avg.rename('atravg'), left_index=True, right_index=True)
    df = df.merge(atri.rename('atr'),left_index=True, right_index=True)
    return df

In [12]:

def greenExplosive(explosive,df):
#Green explosive
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":1,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base]:
            obj.append({"l":2,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":3,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and checkfullbar(0) and checkfullbar(1) and high[2]-low[2] < high[1]-low[1] and checkbarexp(1) and close > open and open[2] < fibpriceind(0.5,1) and close[2] < fibpriceind(0.5,1)
    return obj

In [13]:
def find_nearest_dist_to_zero(data):
    results = []
    for token_item in data:
        nearest_dist = float('inf')
        nearest_zone = None

        for zone_item in token_item['zone']:
            dist = zone_item['dist']

            # Skip non-negative 'dist' values to focus on negative ones
            if dist >= 0:
                continue

            # Calculate absolute difference for nearest to zero comparison
            abs_diff = abs(dist)

            # Update nearest_dist and nearest_zone if closer to zero
            if abs_diff < nearest_dist:
                nearest_dist = abs_diff
                nearest_zone = zone_item

        results.append({
            'token': token_item['token'],
            'nearest_dist': nearest_dist,
            'nearest_zone': nearest_zone
        })

    return results


In [14]:
def createWeeklyBars(i):
    df = createDffromData(data[i][tiks[i]])
    r_df = df.iloc[::-1].reset_index(drop=True)
    mondays, fridays = get_mondays_fridays('2023-02-20', '2024-02-21')
    r_df['date'] = pd.to_datetime(r_df['date'])
    obj = []
    week = []
    openn = []
    close = []
    candle = []
    for j in range(0,len(mondays)-1):
        for i in range(0,len(r_df['date'])-1):
            if r_df['date'][i] >= mondays[j] and r_df['date'][i] <= fridays[j]:
                obj.append({"date":r_df['date'][i],"o":r_df['open'][i],"h":r_df['high'][i],"l":r_df['low'][i],"c":r_df['close'][i]})

        week.append({str(obj[0]['date']):obj})
        obj = []
    
    wdate = []
    wopen = []
    wclose = []
    whigh = []
    wlow = []

    for i in range(0,len(week)-1):
        high = []
        low = []
        wdate.append(", ".join(week[i].keys()).split(' ')[0])
        wclose.append(week[i][", ".join(week[i].keys())][0]['c'])
        wopen.append(week[i][", ".join(week[i].keys())][-1]['o'])
        for j in range(0,len(week[i][", ".join(week[i].keys())])-1):
            high.append(week[i][", ".join(week[i].keys())][j]['h'])
            low.append(week[i][", ".join(week[i].keys())][j]['l'])
        whigh.append(max(high))
        wlow.append(min(low))
        
    wdf = pd.DataFrame()
    wdf['date'] = wdate
    wdf['open'] = wopen
    wdf['high'] = whigh
    wdf['low'] = wlow
    wdf['close'] = wclose
    return wdf


# Main Code

In [15]:
def convert_to_dict(data):
    result = {}
    for item in data:
        key, value = item.split("/")
        result[key] = value
    return result

# Example usage
data =["ABB/3329", "ACC/5633", "ADANIGREEN/912129", "AARTIIND/1793", "AMBUJACEM/325121",
       "ATGL/1552897", "AWL/2076161", "BAJAJHLDNG/78081", "BANKBARODA/1195009", "BEL/98049",
       "BERGEPAINT/103425", "CANBK/2763265", "CHOLAFIN/175361", "COLPAL/3876097", "DABUR/197633",
       "DLF/3771393", "DMART/5097729", "GAIL/1207553", "GODREJCP/2585345", "HAL/589569",
       "HAVELLS/2513665", "HDFCAMC/1086465", "ICICIGI/5573121", "ICICIPRULI/4774913", "INDIGO/2865921",
       "INDUSTOWER/7458561", "IOC/415745", "IRCTC/3484417", "JINDALSTEL/1723649", "LICI/2426881",
       "MARICO/1041153", "MCDOWELL-N/2674433", "MOTHERSON/1076225", "MUTHOOTFIN/6054401", "NAUKRI/3520257",
       "NYKAA/1675521", "PIDILITIND/681985", "PIIND/6191105", "SBICARD/4600577", "SIEMENS/806401",
       "SRF/837889", "TATAPOWER/877057", "TORNTPHARM/900609", "VBL/4843777", "VEDL/784129", "ZOMATO/1304833"]

# your full data here
dictionary = convert_to_dict(data)

a = {"NiftyNxt50":dictionary}


In [16]:
keys = []
values = []

for key, value in a['NiftyNxt50'].items():
    keys.append(key)
    values.append(value)

print("Keys:", keys)
print("Values:", values)

Keys: ['ABB', 'ACC', 'ADANIGREEN', 'AARTIIND', 'AMBUJACEM', 'ATGL', 'AWL', 'BAJAJHLDNG', 'BANKBARODA', 'BEL', 'BERGEPAINT', 'CANBK', 'CHOLAFIN', 'COLPAL', 'DABUR', 'DLF', 'DMART', 'GAIL', 'GODREJCP', 'HAL', 'HAVELLS', 'HDFCAMC', 'ICICIGI', 'ICICIPRULI', 'INDIGO', 'INDUSTOWER', 'IOC', 'IRCTC', 'JINDALSTEL', 'LICI', 'MARICO', 'MCDOWELL-N', 'MOTHERSON', 'MUTHOOTFIN', 'NAUKRI', 'NYKAA', 'PIDILITIND', 'PIIND', 'SBICARD', 'SIEMENS', 'SRF', 'TATAPOWER', 'TORNTPHARM', 'VBL', 'VEDL', 'ZOMATO']
Values: ['3329', '5633', '912129', '1793', '325121', '1552897', '2076161', '78081', '1195009', '98049', '103425', '2763265', '175361', '3876097', '197633', '3771393', '5097729', '1207553', '2585345', '589569', '2513665', '1086465', '5573121', '4774913', '2865921', '7458561', '415745', '3484417', '1723649', '2426881', '1041153', '2674433', '1076225', '6054401', '3520257', '1675521', '681985', '6191105', '4600577', '806401', '837889', '877057', '900609', '4843777', '784129', '1304833']


In [31]:
tiks = []
b = []
for i in range(10,11):
    b.append(keys[i])
    tiks.append(values[i])

In [32]:
#tiks = [6401,3861249,40193,60417,1510401,4267265,4268801,81153,2714625,134657,140033,177665,5215745,2800641,225537,232961,315393,1850625,341249,119553,345089]
#tiks = [1793,3529217,3394561,3930881,951809,4278529,486657,7712001,3484417,2955009,3660545,5728513,2455041,2763265]
#tiks = [5097729]
data = []
for i in range(0,len(tiks)):
    data.append({tiks[i]:fetchData(str(tiks[i]),'2023-02-21','2024-02-21')})
#data = fetchData('160769','2023-02-14','2024-02-14')

In [156]:
df = createDffromData(data[0][tiks[0]])
r_df = df.iloc[::-1].reset_index(drop=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11268\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]


In [157]:
mondays, fridays = get_mondays_fridays('2023-02-20', '2024-02-21')
print("Mondays:")
print(mondays)
print("\nFridays:")
print(fridays)

Mondays:
DatetimeIndex(['2023-02-20', '2023-02-27', '2023-03-06', '2023-03-13',
               '2023-03-20', '2023-03-27', '2023-04-03', '2023-04-10',
               '2023-04-17', '2023-04-24', '2023-05-01', '2023-05-08',
               '2023-05-15', '2023-05-22', '2023-05-29', '2023-06-05',
               '2023-06-12', '2023-06-19', '2023-06-26', '2023-07-03',
               '2023-07-10', '2023-07-17', '2023-07-24', '2023-07-31',
               '2023-08-07', '2023-08-14', '2023-08-21', '2023-08-28',
               '2023-09-04', '2023-09-11', '2023-09-18', '2023-09-25',
               '2023-10-02', '2023-10-09', '2023-10-16', '2023-10-23',
               '2023-10-30', '2023-11-06', '2023-11-13', '2023-11-20',
               '2023-11-27', '2023-12-04', '2023-12-11', '2023-12-18',
               '2023-12-25', '2024-01-01', '2024-01-08', '2024-01-15',
               '2024-01-22', '2024-01-29', '2024-02-05', '2024-02-12',
               '2024-02-19'],
              dtype='datetime64[ns]', 

In [158]:
r_df['date'] = pd.to_datetime(r_df['date'])

In [159]:
obj = []
week = []
openn = []
close = []
candle = []
for j in range(0,len(mondays)-1):
    for i in range(0,len(r_df['date'])-1):
        if r_df['date'][i] >= mondays[j] and r_df['date'][i] <= fridays[j]:
            obj.append({"date":r_df['date'][i],"o":r_df['open'][i],"h":r_df['high'][i],"l":r_df['low'][i],"c":r_df['close'][i]})
           
    week.append({str(obj[0]['date']):obj})
    obj = []


In [160]:
wdate = []
wopen = []
wclose = []
whigh = []
wlow = []

for i in range(0,len(week)-1):
    high = []
    low = []
    wdate.append(", ".join(week[i].keys()))
    wclose.append(week[i][", ".join(week[i].keys())][0]['c'])
    wopen.append(week[i][", ".join(week[i].keys())][-1]['o'])
    for j in range(0,len(week[i][", ".join(week[i].keys())])-1):
        high.append(week[i][", ".join(week[i].keys())][j]['h'])
        low.append(week[i][", ".join(week[i].keys())][j]['l'])
    whigh.append(max(high))
    wlow.append(min(low))

In [173]:
wdf = pd.DataFrame()
wdf['date'] = wdate
wdf['open'] = wopen
wdf['high'] = whigh
wdf['low'] = wlow
wdf['close'] = wclose
wdf

date     open     high      low    close
0   2023-02-24 00:00:00  3502.00  3515.00  3437.00  3499.20
1   2023-03-03 00:00:00  3485.05  3508.00  3373.00  3463.25
2   2023-03-10 00:00:00  3480.60  3453.00  3380.05  3391.85
3   2023-03-17 00:00:00  3391.85  3455.15  3292.00  3311.30
4   2023-03-24 00:00:00  3295.00  3415.00  3326.15  3358.40
5   2023-03-31 00:00:00  3367.00  3415.00  3300.00  3401.05
6   2023-04-06 00:00:00  3413.05  3666.70  3480.10  3496.10
7   2023-04-13 00:00:00  3504.00  3518.95  3446.00  3501.05
8   2023-04-21 00:00:00  3500.00  3565.00  3440.00  3455.90
9   2023-04-28 00:00:00  3469.80  3522.00  3443.05  3511.95
10  2023-05-05 00:00:00  3515.00  3605.00  3516.05  3598.20
11  2023-05-12 00:00:00  3615.10  3750.00  3622.65  3677.55
12  2023-05-19 00:00:00  3550.00  3557.05  3352.00  3395.25
13  2023-05-26 00:00:00  3400.00  3512.00  3406.20  3501.75
14  2023-06-02 00:00:00  3540.00  3555.00  3437.35  3531.60
15  2023-06-09 00:00:00  3550.00  3648.00  3495.05  3632.10
16  2023-06-16 00:00:00  3646.95  4144.00  3630.00  4085.60
17  2023-06-23 00:00:00  4086.00  4045.00  3731.90  3744.05
18  2023-06-30 00:00:00  3744.05  3988.00  3841.00  3889.20
19  2023-07-07 00:00:00  3889.20  3929.95  3800.00  3808.95
20  2023-07-14 00:00:00  3810.00  3881.85  3772.05  3838.75
21  2023-07-21 00:00:00  3796.85  3720.00  3643.60  3675.70
22  2023-07-28 00:00:00  3676.00  3732.15  3660.05  3713.90
23  2023-08-04 00:00:00  3713.05  3779.90  3640.00  3652.10
24  2023-08-11 00:00:00  3661.90  3686.70  3545.00  3550.05
25  2023-08-18 00:00:00  3567.95  3594.15  3491.25  3540.95
26  2023-08-25 00:00:00  3540.00  3582.95  3530.05  3534.60
27  2023-09-01 00:00:00  3533.00  3765.00  3615.00  3761.05
28  2023-09-08 00:00:00  3771.95  3856.80  3727.70  3782.40
29  2023-09-15 00:00:00  3791.70  3850.00  3731.20  3796.00
30  2023-09-22 00:00:00  3795.80  3789.95  3653.95  3689.10
31  2023-09-29 00:00:00  3700.00  3698.00  3625.00  3675.60
32  2023-10-06 00:00:00  3675.40  3938.40  3760.00  3833.95
33  2023-10-13 00:00:00  3809.60  3941.00  3775.55  3935.85
34  2023-10-20 00:00:00  3825.05  3885.00  3742.10  3749.50
35  2023-10-27 00:00:00  3750.05  3758.00  3620.10  3665.45
36  2023-11-03 00:00:00  3686.70  3677.95  3620.00  3649.70
37  2023-11-10 00:00:00  3654.55  3800.00  3636.00  3798.40
38  2023-11-17 00:00:00  3809.95  3843.00  3750.10  3810.90
39  2023-11-24 00:00:00  3810.90  3873.85  3796.00  3842.90
40  2023-12-01 00:00:00  3815.00  3995.00  3834.00  3988.40
41  2023-12-08 00:00:00  4000.00  4206.00  3949.00  4070.65
42  2023-12-15 00:00:00  4070.65  4100.00  3945.10  4045.65
43  2023-12-22 00:00:00  4045.00  4048.95  3905.00  4007.95
44  2023-12-29 00:00:00  4009.95  4098.90  3984.20  4082.65
45  2024-01-05 00:00:00  4075.00  4155.00  3810.00  3863.50
46  2024-01-12 00:00:00  3893.95  3850.00  3787.00  3841.00
47  2024-01-19 00:00:00  3924.95  3865.00  3704.50  3731.70
48  2024-01-25 00:00:00  3675.00  3750.00  3646.05  3734.25
49  2024-02-02 00:00:00  3700.00  3826.60  3652.75  3784.30
50  2024-02-09 00:00:00  3785.00  3787.95  3699.00  3719.20

In [174]:
atri = atr(wdf,1)
atr_avg = atr(wdf,20)

In [175]:
wdf = wdf.merge(atr_avg.rename('atravg'), left_index=True, right_index=True)
wdf = wdf.merge(atri.rename('atr'),left_index=True, right_index=True)

In [184]:
wdf

date     open     high      low    close      atravg  \
0   2023-02-24 00:00:00  3502.00  3515.00  3437.00  3499.20   78.000000   
1   2023-03-03 00:00:00  3485.05  3508.00  3373.00  3463.25   80.850000   
2   2023-03-10 00:00:00  3480.60  3453.00  3380.05  3391.85   80.967500   
3   2023-03-17 00:00:00  3391.85  3455.15  3292.00  3311.30   85.076625   
4   2023-03-24 00:00:00  3295.00  3415.00  3326.15  3358.40   86.007794   
5   2023-03-31 00:00:00  3367.00  3415.00  3300.00  3401.05   87.457404   
6   2023-04-06 00:00:00  3413.05  3666.70  3480.10  3496.10   96.367034   
7   2023-04-13 00:00:00  3504.00  3518.95  3446.00  3501.05   95.196182   
8   2023-04-21 00:00:00  3500.00  3565.00  3440.00  3455.90   96.686373   
9   2023-04-28 00:00:00  3469.80  3522.00  3443.05  3511.95   95.799554   
10  2023-05-05 00:00:00  3515.00  3605.00  3516.05  3598.20   95.662077   
11  2023-05-12 00:00:00  3615.10  3750.00  3622.65  3677.55   98.468973   
12  2023-05-19 00:00:00  3550.00  3557.05  3352.00  3395.25  109.823024   
13  2023-05-26 00:00:00  3400.00  3512.00  3406.20  3501.75  110.169373   
14  2023-06-02 00:00:00  3540.00  3555.00  3437.35  3531.60  110.543404   
15  2023-06-09 00:00:00  3550.00  3648.00  3495.05  3632.10  112.663734   
16  2023-06-16 00:00:00  3646.95  4144.00  3630.00  4085.60  132.730547   
17  2023-06-23 00:00:00  4086.00  4045.00  3731.90  3744.05  143.779020   
18  2023-06-30 00:00:00  3744.05  3988.00  3841.00  3889.20  148.787569   
19  2023-07-07 00:00:00  3889.20  3929.95  3800.00  3808.95  147.845691   
20  2023-07-14 00:00:00  3810.00  3881.85  3772.05  3838.75  145.943406   
21  2023-07-21 00:00:00  3796.85  3720.00  3643.60  3675.70  148.403736   
22  2023-07-28 00:00:00  3676.00  3732.15  3660.05  3713.90  144.588549   
23  2023-08-04 00:00:00  3713.05  3779.90  3640.00  3652.10  144.354122   
24  2023-08-11 00:00:00  3661.90  3686.70  3545.00  3550.05  144.221415   
25  2023-08-18 00:00:00  3567.95  3594.15  3491.25  3540.95  142.155345   
26  2023-08-25 00:00:00  3540.00  3582.95  3530.05  3534.60  137.692577   
27  2023-09-01 00:00:00  3533.00  3765.00  3615.00  3761.05  142.327949   
28  2023-09-08 00:00:00  3771.95  3856.80  3727.70  3782.40  141.666551   
29  2023-09-15 00:00:00  3791.70  3850.00  3731.20  3796.00  140.523224   
30  2023-09-22 00:00:00  3795.80  3789.95  3653.95  3689.10  140.599562   
31  2023-09-29 00:00:00  3700.00  3698.00  3625.00  3675.60  137.219584   
32  2023-10-06 00:00:00  3675.40  3938.40  3760.00  3833.95  143.498605   
33  2023-10-13 00:00:00  3809.60  3941.00  3775.55  3935.85  144.596175   
34  2023-10-20 00:00:00  3825.05  3885.00  3742.10  3749.50  147.053866   
35  2023-10-27 00:00:00  3750.05  3758.00  3620.10  3665.45  146.596173   
36  2023-11-03 00:00:00  3686.70  3677.95  3620.00  3649.70  142.163864   
37  2023-11-10 00:00:00  3654.55  3800.00  3636.00  3798.40  143.255671   
38  2023-11-17 00:00:00  3809.95  3843.00  3750.10  3810.90  140.737887   
39  2023-11-24 00:00:00  3810.90  3873.85  3796.00  3842.90  137.593493   
40  2023-12-01 00:00:00  3815.00  3995.00  3834.00  3988.40  138.763818   
41  2023-12-08 00:00:00  4000.00  4206.00  3949.00  4070.65  144.675627   
42  2023-12-15 00:00:00  4070.65  4100.00  3945.10  4045.65  145.186846   
43  2023-12-22 00:00:00  4045.00  4048.95  3905.00  4007.95  145.125004   
44  2023-12-29 00:00:00  4009.95  4098.90  3984.20  4082.65  143.603754   
45  2024-01-05 00:00:00  4075.00  4155.00  3810.00  3863.50  153.673566   
46  2024-01-12 00:00:00  3893.95  3850.00  3787.00  3841.00  149.814888   
47  2024-01-19 00:00:00  3924.95  3865.00  3704.50  3731.70  150.349143   
48  2024-01-25 00:00:00  3675.00  3750.00  3646.05  3734.25  148.029186   
49  2024-02-02 00:00:00  3700.00  3826.60  3652.75  3784.30  149.320227   
50  2024-02-09 00:00:00  3785.00  3787.95  3699.00  3719.20  146.301715   

       atr  
0    78.00  
1   135.00  
2    83.20  
3   163.15  
4   103.70  
5   115.00  
6   265.65  
7    72.95  
8

In [197]:
explosive = checkdata(wdf)

In [178]:
greenExplosive(explosive,wdf)

[{'l': 1,
  'legoutDate': '2023-06-09 00:00:00',
  'baseHigh': 3555.0,
  'baseLow': 3437.35,
  'dist': -7.566302652106084},
 {'l': 1,
  'legoutDate': '2023-06-16 00:00:00',
  'baseHigh': 3648.0,
  'baseLow': 3495.05,
  'dist': -5.14820592823713},
 {'l': 2,
  'legoutDate': '2023-10-06 00:00:00',
  'baseHigh': 3698.0,
  'baseLow': 3625.0,
  'dist': -3.848153926157046}]

In [199]:
for i in range(0,len(explosive['Date'])-1):
    base = explosive['index'][i]-1
    print(explosive['Date'][i])
    if explosive['Close'][i] > explosive['Open'][i] and abs(wdf['close'][base]-wdf['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.4,i):
        print(explosive['Date'][i],wdf['high'][base],wdf['low'][base],((wdf['high'][base]-ltp)/ltp)*100)
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
    elif explosive['Close'][i] > explosive['Open'][i] and wdf['open'][base] > explosive['Open'][i] and wdf['close'][base] < explosive['Low'][i] and wdf['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > wdf['high'][base]-wdf['low'][base]:
        print(explosive['Date'][i],wdf['high'][base],wdf['low'][base],((wdf['high'][base]-ltp)/ltp)*100)
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
    elif explosive['Close'][i] > explosive['Open'][i] and wdf['close'][base] < fiblvlprice(0.95,i) and wdf['open'][base] > fiblvlprice(1.25,i) and wdf['close'][base] > wdf['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
        print(explosive['Date'][i],wdf['high'][base],wdf['low'][base],((wdf['high'][base]-ltp)/ltp)*100)
    elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
        print(explosive['Date'][i],wdf['high'][base],wdf['low'][base],((wdf['high'][base]-ltp)/ltp)*100)

2023-03-17 00:00:00
2023-03-24 00:00:00
2023-03-31 00:00:00
2023-04-06 00:00:00
2023-04-21 00:00:00
2023-05-12 00:00:00
2023-05-19 00:00:00
2023-06-09 00:00:00
2023-06-09 00:00:00 3555.0 3437.35 -7.566302652106084
2023-06-16 00:00:00
2023-06-16 00:00:00 3648.0 3495.05 -5.14820592823713
2023-06-23 00:00:00
2023-09-01 00:00:00
2023-09-01 00:00:00 3582.95 3530.05 -6.839573582943323
2023-10-06 00:00:00
2023-10-06 00:00:00 3698.0 3625.0 -3.848153926157046
2023-10-13 00:00:00
2023-11-10 00:00:00
2023-12-01 00:00:00
2023-12-01 00:00:00 3873.85 3796.0 0.724128965158604
2023-12-08 00:00:00
2024-01-05 00:00:00
2024-01-19 00:00:00


In [198]:
explosive

index                 Date     Open     High      Low    Close
0       3  2023-03-17 00:00:00  3391.85  3455.15  3292.00  3311.30
1       4  2023-03-24 00:00:00  3295.00  3415.00  3326.15  3358.40
2       5  2023-03-31 00:00:00  3367.00  3415.00  3300.00  3401.05
3       6  2023-04-06 00:00:00  3413.05  3666.70  3480.10  3496.10
4       8  2023-04-21 00:00:00  3500.00  3565.00  3440.00  3455.90
5      11  2023-05-12 00:00:00  3615.10  3750.00  3622.65  3677.55
6      12  2023-05-19 00:00:00  3550.00  3557.05  3352.00  3395.25
7      15  2023-06-09 00:00:00  3550.00  3648.00  3495.05  3632.10
8      16  2023-06-16 00:00:00  3646.95  4144.00  3630.00  4085.60
9      17  2023-06-23 00:00:00  4086.00  4045.00  3731.90  3744.05
10     27  2023-09-01 00:00:00  3533.00  3765.00  3615.00  3761.05
11     32  2023-10-06 00:00:00  3675.40  3938.40  3760.00  3833.95
12     33  2023-10-13 00:00:00  3809.60  3941.00  3775.55  3935.85
13     37  2023-11-10 00:00:00  3654.55  3800.00  3636.00  3798.40
14     40  2023-12-01 00:00:00  3815.00  3995.00  3834.00  3988.40
15     41  2023-12-08 00:00:00  4000.00  4206.00  3949.00  4070.65
16     45  2024-01-05 00:00:00  4075.00  4155.00  3810.00  3863.50
17     47  2024-01-19 00:00:00  3924.95  3865.00  3704.50  3731.70
18     49  2024-02-02 00:00:00  3700.00  3826.60  3652.75  3784.30

# Main Code

In [38]:

dmzs = []
for j in range(0,len(tiks)):
    df = createWeeklyBars(j)
    ltp = df['close'].iloc[-1]
    print(ltp)
    print(tiks[j])
    df = putAtrValues(df)
    explosive1 = checkdata(df)
    explosive = explosive1.iloc[::-1].reset_index(drop=True)
    #print(explosive)
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.7,i) and df['open'][base] < fiblvlprice(0.5,i) and df['close'][base] < fiblvlprice(0.5,i) and explosive['Open'][i] < fiblvlprice(0.3,i) and df['high'][base] < fiblvlprice(0.5,i):
            obj.append(
                {
                    "l":1,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base] and (explosive['Close'][i]-explosive['Close'][i])*0.25 > (df['open'][base]-df['close'][base]):
            obj.append(
                {
                    "l":2,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append(
                {
                    "l":3,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
            obj.append(
                {
                    "l":4,
                    "legoutDate":explosive['Date'][i],
                    "baseHigh":df['high'][base],
                    "baseLow":df['low'][base],
                    "dist":((df['high'][base]-ltp)/ltp)*100,
                    "ltp":ltp,
                    "chartData":{
                        "series":[
                            {
                                "data":
                                [
                                        {
                                "x":df['date'][base],
                                "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                            },
                            {
                                "x":explosive['Date'][i],
                                "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                            }
                                ]
                            }
                        ],
                        "legoutDate":explosive['Date'][i]
                    }
                })
            
    dmzs.append({tiks[j]:obj})

C:\Users\Admin\AppData\Local\Temp\ipykernel_2180\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]


554.0
103425


In [39]:
#dmzs

In [40]:
zones = []
for ti in range(0,len(data)):
    brokenzone = []
    df = createDffromData(data[ti][tiks[ti]])
    for i in range (0,len(dmzs[ti][tiks[ti]])):
        for j in range(0,len(df)):
            #print(df['date'][j] , dmzs[ti][tiks[ti]][i]['chartData']['legoutDate'])
            if df['date'][j] == dmzs[ti][tiks[ti]][i]['chartData']['legoutDate']:
                #print(df['date'][j])
                for k in range(j+1,len(df)):
                    #print(k,df['date'][k],df['low'][k],dmzs[0][tiks[0]][i]['baseLow'])
                    if dmzs[ti][tiks[ti]][i]['baseLow'] > df['low'][k]:
                        #print('dfdf')#print(dmzs[ti][tiks[ti]][i]['legoutDate'],'broken',dmzs[ti][tiks[ti]][i]['baseLow'],df['date'][k])
                        brokenzone.append(dmzs[ti][tiks[ti]][i]['chartData']['legoutDate'])
                        #print(brokenzone)
                        break
    a = dmzs[ti][tiks[ti]]
    bz_set = set(brokenzone)
    #print(tiks[ti])
    # Efficiently remove matching dictionaries from the end
    for i in range(len(a) - 1, -1, -1):
        if a[i]['legoutDate'] in bz_set:
            del a[i]
    #print(json.dumps(a,indent=4))
    zones.append({'ticker':b[ti],'token':tiks[ti],'zone':a})

C:\Users\Admin\AppData\Local\Temp\ipykernel_2180\1498270930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i]=df['date'][i].split('T')[0]


In [41]:
zones

[{'ticker': 'BERGEPAINT', 'token': '103425', 'zone': []}]

In [42]:
#tik = 1
for i in range(0,len(zones)):
    if len(zones[i]['zone']) > 0:
        print(mycollection.insert_one(zones[i]))